In [ ]:
''' VMP 2022-03-05: 
based on Analysis_RASMUS.ipynb
some changes to make it work & some warnings.
double-check with Nielbo. 
'''

In [2]:
#### Imports
import pandas as pd 
import pickle as pkl 
import altair as alt
import numpy as np 
import os 

# setting up matplotlib settings
# Source: https://towardsdatascience.com/making-matplotlib-beautiful-by-default-d0d41e3534fd
import matplotlib.pyplot as plt
import seaborn as sns 
import matplotlib.font_manager as font_manager

#%matplotlib inline
from pandas.plotting import scatter_matrix
import seaborn as sns
sns.set(style="whitegrid")
import re
import pyLDAvis

import matplotlib.dates as mdates
import matplotlib.pyplot as plt

# font
font_dirs = ['/Library/Fonts', ]
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
font_list = font_manager.createFontList(font_files)
font_manager.fontManager.ttflist.extend(font_list)

plt.rcParams['font.family'] = 'DIN Condensed Bold'

# set matplotlib aesthetics
CB91_Blue = '#2CBDFE'
CB91_Green = '#47DBCD'
CB91_Pink = '#F3A0F2'
CB91_Purple = '#9D2EC5'
CB91_Violet = '#661D98'
CB91_Amber = '#F5B14C'

color_list = [CB91_Blue, CB91_Pink, CB91_Green, CB91_Amber,
              CB91_Purple, CB91_Violet]

plt.rcParams['axes.prop_cycle'] = plt.cycler(color=color_list)

sns.set(rc={
            'axes.axisbelow': False,
            'axes.edgecolor': 'lightgrey',
            'axes.facecolor': 'None',
            'axes.grid': False,
            'axes.labelcolor': 'dimgrey',
            'axes.spines.right': False,
            'axes.spines.top': False,
            'figure.facecolor': 'white',
            'lines.solid_capstyle': 'round',
            'patch.edgecolor': 'w',
            'patch.force_edgecolor': True,
            'text.color': 'dimgrey',
            'xtick.bottom': False,
            'xtick.color': 'dimgrey',
            'xtick.direction': 'out',
            'xtick.top': False,
            'ytick.color': 'dimgrey',
            'ytick.direction': 'out',
            'ytick.left': False,
            'ytick.right': False,
            'savefig.dpi': 800})

#plt.rcParams["savefig.dpi"] = 'figure'
sns.set_context("notebook", rc={"font.size":12,
                                "axes.titlesize":16,
                                "axes.labelsize":16})


#alt.data_transformers.disable_max_rows()

/home/cnsome/china_twitter/lib/python3.6/site-packages/ipykernel_launcher.py:27: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.


# Load data

In [3]:
# function which is basically the same as in "helper_functions" but manages new paths 
def prep_data():
    """Preparing the data

    Returns:
        pd.Dataframes: Returns three dataframes - the whole dataset, the diplomats and the media.
    """    
    en_df = pd.read_csv("../data/all_from_clean.csv")

    en_df['month'] = pd.DatetimeIndex(en_df['created_at']).month
    en_df['date'] = pd.DatetimeIndex(en_df['created_at']).date
    
    
    def retweet_binary(string):
        if string == "retweeted":
            return "Retweet"
        else:
            return "Other"

    en_df["retweet_bin"] = en_df["retweet"].apply(lambda x: retweet_binary(x))
    
    en_df["date"] = pd.to_datetime(en_df["date"])
    
    return en_df, en_df[en_df["category"] == "Diplomat"].reset_index(drop=True), en_df[en_df["category"] == "Media"].reset_index(drop=True)
    

In [4]:
data, diplomats, media = prep_data()

# Load models

In [5]:
def load_models():
    with open("../data/models/Media_LDA.pkl", "rb") as f:
        media = pkl.load(f)
    
    with open("../data/models/Diplomat_LDA.pkl", "rb") as f:
        diplo = pkl.load(f)
        
    return media, diplo

In [6]:
media_dict, diplo_dict = load_models()

/home/cnsome/china_twitter/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/home/cnsome/china_twitter/lib/python3.6/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [7]:
# diplomats without retweets
diplo_without = diplomats[diplomats["retweet"] != "retweeted"].reset_index(drop = True)

In [11]:
# VMP 2022-03-05
# for the below I have just used the code in Analysis_RASMUS.ipynb
# not sure whether diplo_dict() should be filtered for retweets?
# made some changes

In [18]:
# Task 1: Diplomat document representation and metadata
## add $\theta$ to to diplomats

## comment0: diplo_without, model is trained without retweet to avoid redundancy
## comment1: corpus is filtered for extremes with Gensim 
## comment2 (VMP 2022-03-05): using max_topics, rather than len(theta[0]). 

# predict theta_i on document in corpus 
print(f'\n[INFO] predicting \u03B8...\n{"*"*25}')
verbose = 5000
theta = list()
for i, doc in enumerate(diplo_dict['corpus']):
    vector = diplo_dict['model'][doc]
    theta.append(vector[0])
    if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
        print(f'[INFO] processed {i + 1}/{len(diplo_dict["corpus"])}')
# dimension consistent theta array
ntopics_lst = [len(doc) for doc in theta]
max_topics = max(ntopics_lst)
print(f"maximum number of topics: {max_topics}")
(m, k) = (len(theta), max_topics) #kappa VMP2022-03-05: (previously: (len(theta), len(theta[0])))
theta_transform = np.zeros((m, k))
for (i, doc) in enumerate(theta):
    theta_i = np.zeros((1,30)) # VMP: where is this used?
    for j, var in doc:
        theta_transform[i,j] = var
# write to csv
df = pd.DataFrame()
for i in range(len(theta_transform[0,:])):
    df[f'var {i}'] = theta_transform[:,i]
dfout = pd.concat([diplo_without, df], axis=1)
fname = os.path.join('..', 'data', 'diplomats_noretweet_theta_df.csv')
dfout.to_csv(fname, index=False)
print(f'{"*"*25}\n[INFO] data exported to {fname}')
fname_maxqda = '..' + fname.split('.')[2] + '.xlsx' # VMP2022-03-05: (previously: fname.split('.')[0] + '.xlsx')
dfout['created_at'] = dfout['created_at'].astype(str)
dfout.to_excel(fname_maxqda, index=False)
print(f'[INFO] data exported to {fname_maxqda}')


[INFO] predicting θ...
*************************
[INFO] processed 5000/25830
[INFO] processed 10000/25830
[INFO] processed 15000/25830
[INFO] processed 20000/25830
[INFO] processed 25000/25830
maximum number of topics: 30
*************************
[INFO] data exported to ../data/diplomats_noretweet_theta_df.csv
[INFO] data exported to ../data/diplomats_noretweet_theta_df.xlsx


# Trouble-shooting

In [ ]:
### trouble-shoot the problem (IndexError: --> theta_transform[i,j] = var)
print(len(theta)) # 25.830 elements
print(len(theta[0])) # length 8 for the first element -- issue is that some elements are longer. 
print(theta_transform.shape) # and this is exactly what we specify

In [10]:
### find "longest" list in theta (topics)
# this is the issue -- the longest list has 30 elements 
ntopics_lst = [len(doc) for doc in theta]
max_topics = max(ntopics_lst)
print(max_topics)

30
